# Supplementing predictions eith fallback tables for low vol tld-re's

In [1]:
options(repr.matrix.max.cols=50, repr.matrix.max.rows=100)

In [2]:
# install.packages("pkgcond")

In [3]:
library(dplyr)
library(data.table)
library(partykit)
library(tictoc)
library(caret)
library(e1071)
library(randomForest)
library(ranger)

#for 3d plotting
library(akima)
library(plotly)

# for prep data
# library(rPython)
library(stringr)
library(pbapply)
library(stringdist)
library(data.table)
library(dominanceanalysis)
library(zoo)


getwd()


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


Loading required package: grid

Loading required package: libcoin

Loading required package: mvtnorm

Loading required package: lattice

Loading required package: ggplot2

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:ggplot2’:

    margin


The following object is masked from ‘package:dplyr’:

    combine



Attaching package: ‘ranger’


The following object is masked from ‘package:randomForest’:

    importance



Attaching package: ‘plotly’


The following object is masked from ‘package:ggplot2’:

    last_plot


The following object is masked from ‘package:sta

[1] "/home/jupyter/Domains_202003/scripts/phaseII_06_fallbacksupp"

In [4]:
# install.packages(c("zoo"))

In [5]:

source('../orig/functions.R')

source('../orig/functions_models.R')

source('../phaseII_03_forest/functions_eval.R')



Attaching package: ‘tidyr’


The following object is masked from ‘package:stringdist’:

    extract




In [39]:
source('functions_metalearning.R')

In [60]:
source('functions_fallback.R')

In [8]:
# source('load_prep_data_expiry_2.R') #read in train test and predictions manually instead

# READ CSV's
expiry_df_test_preds <- read.csv("../../data/output/datapull_20201116/expiry_df_test_preds.csv")
expiry_df_train <- read.csv("../../data/output/datapull_20201116/expiry_df_train.csv")
expiry_df_test <- read.csv("../../data/output/datapull_20201116/expiry_df_test.csv")

# expiry_df
# expiry_train_df 
# expiry_test_df 
# expiry_list
# expiry_train_list
# expiry_test_list

In [15]:
dim(expiry_df_train)
dim(expiry_df_test_preds)

[1] 3729384      40

[1] 932347     40

# APPLY fallback functions

In [11]:
# geo_suppl for train and test-pred data
geoLookupDF <- read.csv("/home/jupyter/Domains_202003/data/input/PredictiveModelAnalysis_ResellerGeoMap.csv")

In [13]:
expiry_df_train_g <- geo_suppl(expiry_df_train, geoLookupDF = geoLookupDF)
expiry_df_test_preds_g <- geo_suppl(expiry_df_test_preds, geoLookupDF = geoLookupDF)

Expiry data originally has 3729384 rows and 3729384 missing geo's.
... after intial merge on reseller & _country, expiry has 3729384 rows and 3726455 missing geo's.
... after secondary fill with _country, expiry has 3729384 rows and 6983 missing geo's.
... after manual tweaks with _country, expiry has 3729384 rows and 29 missing geo's.
Expiry data originally has 932347 rows and 932347 missing geo's.
... after intial merge on reseller & _country, expiry has 932347 rows and 931606 missing geo's.
... after secondary fill with _country, expiry has 932347 rows and 1741 missing geo's.
... after manual tweaks with _country, expiry has 932347 rows and 8 missing geo's.


In [14]:
# ensure same number of dims as before
dim(expiry_df_train_g)
dim(expiry_df_test_preds_g)

[1] 3729384      41

[1] 932347     41

In [16]:
# Examine remaining NA reseller_geos
expiry_df_train_g %>% filter(is.na(reseller_geo)) %>% distinct(reseller,reseller_country, reseller_geo)
expiry_df_test_preds_g %>% filter(is.na(reseller_geo)) %>% distinct(reseller,reseller_country, reseller_geo)

reseller,reseller_country,reseller_geo
<fct>,<fct>,<fct>
AnyGaming,,NA
Beijing Brandma,,NA
Innovadeus,,NA
Net4 India,,NA
NetClues,,NA
OwnRegistrar,,NA
Virtua Drug,,NA


reseller,reseller_country,reseller_geo
<fct>,<fct>,<fct>
,,NA
NetClues,,NA
OwnRegistrar,,NA
Virtua Drug,,NA


In [21]:
str(expiry_df_train_g$creation_date)
expiry_df_train_g$creation_date <- as.Date(expiry_df_train_g$creation_date)
str(expiry_df_train_g$creation_date)

 Factor w/ 974 levels "2017-01-01","2017-01-02",..: 16 650 45 49 49 59 60 83 88 90 ...
 Date[1:3729384], format: "2017-01-16" "2018-10-12" "2017-02-14" "2017-02-18" "2017-02-18" ...


In [22]:
# generate list of fallback tables
npv_fallback_list = fallback_gen( npv_historic_renewal_data = expiry_df_train_g, 
                                 reseller_am_geo_map = geoLookupDF)

`summarise()` regrouping output by 'tld', 'reseller_geo' (override with `.groups` argument)

`summarise()` regrouping output by 'reseller_geo' (override with `.groups` argument)

`summarise()` regrouping output by 'tld' (override with `.groups` argument)

`summarise()` regrouping output by 'tld' (override with `.groups` argument)

`summarise()` ungrouping output (override with `.groups` argument)

`summarise()` regrouping output by 'tld' (override with `.groups` argument)

`summarise()` ungrouping output (override with `.groups` argument)

`summarise()` regrouping output by 'region' (override with `.groups` argument)

`summarise()` ungrouping output (override with `.groups` argument)

`summarise()` ungrouping output (override with `.groups` argument)



In [23]:
# return list members to in-memory objects of the same name
names(npv_fallback_list)
for(i in 1:length(npv_fallback_list)) assign(names(npv_fallback_list)[i], npv_fallback_list[[i]])

[1] "npv_fallback_first_geo_arpt_tld" "npv_fallback_first_geo_arpt"    
 [3] "npv_fallback_first_tld_arpt"     "npv_fallback_second_tld_geo"    
 [5] "npv_fallback_second_geo"         "npv_fallback_third_tld_geo"     
 [7] "npv_fallback_third_geo"          "npv_fallback_first_final"       
 [9] "npv_fallback_second_final"       "npv_fallback_third_final"

In [36]:
# convert pred df to pred list
expiry_list_train_g <- split(expiry_df_train_g, expiry_df_train_g$tld_registrar_index)

In [43]:
length(expiry_list_train_g)

[1] 2109

In [41]:
# generate list of low-volume tld-re's from training data
tld_registrar_excl_list = tld_registrar_excl(train_list = expiry_list_train_g)

Excluding 1706 tld-re's (80.89%), due to volume < 500 
Excluding 157 tld-re's (7.44%) associated with tld's 'pw' and 'in.net' 


In [42]:
length(tld_registrar_excl_list)

[1] 1706

In [ ]:
head(expiry_df_test_preds_g)

In [47]:
# generate placeholder columns in preds df for fall backapp
# ... where predictions for low-volume tld-registrars are NA

expiry_df_test_preds_g2 <- expiry_df_test_preds_g %>%
     mutate( across(contains('pred_'), 
                    .fns = list(fb = ~ifelse(tld_registrar_index %in% tld_registrar_excl_list, NA, . )) ))

In [58]:
expiry_df_test_preds_g2 %>% select(contains('pred_')) %>% head()

,pred_agg_glm_ALL,pred_agg_rf_ALL,pred_agg_glm,pred_agg_rf,pred_seg_glm_ALL,pred_seg_rf_ALL,pred_seg2_glm_ALL,pred_seg2_rf_ALL,pred_agg_glm_ALL_fb,pred_agg_rf_ALL_fb,pred_agg_glm_fb,pred_agg_rf_fb,pred_seg_glm_ALL_fb,pred_seg_rf_ALL_fb,pred_seg2_glm_ALL_fb,pred_seg2_rf_ALL_fb
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,NA,0.9550391,NA,0.7713468,NA,NA,NA,NA,NA,0.9550391,NA,0.7713468,NA,NA,NA,NA
2,0.817,0.6088277,0.817,0.6676114,0.543,0.7268116,0.5644593,0.7075837,0.817,0.6088277,0.817,0.6676114,0.543,0.7268116,0.5644593,0.7075837
3,0.814,0.6477227,0.814,0.6877335,0.536,0.7155560,0.5601541,0.7314602,0.814,0.6477227,0.814,0.6877335,0.536,0.7155560,0.5601541,0.7314602
4,0.833,0.2384536,0.838,0.2588170,0.513,0.2754756,0.1642908,0.1535971,0.833,0.2384536,0.838,0.2588170,0.513,0.2754756,0.1642908,0.1535971
5,0.772,0.3972606,0.775,0.3878439,0.468,0.3962124,0.4979086,0.3769304,0.772,0.3972606,0.775,0.3878439,0.468,0.3962124,0.4979086,0.3769304
6,0.823,0.1925156,0.829,0.1730035,0.516,0.1675698,0.1723292,0.1027711,0.823,0.1925156,0.829,0.1730035,0.516,0.1675698,0.1723292,0.1027711


In [62]:
# apply fallback tables TST
expiry_df_test_preds_g3 <- fallback_app_1(test_data_op=expiry_df_test_preds_g2,
               in_col='pred_seg2_rf_ALL_fb',
               out_col='pred_seg2_rf_ALL_fb2')
expiry_df_test_preds_g3 <- fallback_app_1(test_data_op=expiry_df_test_preds_g3,
               in_col='pred_seg2_glm_ALL_fb',
               out_col='pred_seg2_glm_ALL_fb2')
expiry_df_test_preds_g3 <- fallback_app_1(test_data_op=expiry_df_test_preds_g3,
               in_col='pred_seg_rf_ALL_fb',
               out_col='pred_seg_rf_ALL_fb2')
expiry_df_test_preds_g3 <- fallback_app_1(test_data_op=expiry_df_test_preds_g3,
               in_col='pred_seg_glm_ALL_fb',
               out_col='pred_seg_glm_ALL_fb2')
expiry_df_test_preds_g3 <- fallback_app_1(test_data_op=expiry_df_test_preds_g3,
               in_col='pred_agg_rf_fb',
               out_col='pred_agg_rf_fb2')
expiry_df_test_preds_g3 <- fallback_app_1(test_data_op=expiry_df_test_preds_g3,
               in_col='pred_agg_glm_fb',
               out_col='pred_agg_glm_fb2')
expiry_df_test_preds_g3 <- fallback_app_1(test_data_op=expiry_df_test_preds_g3,
               in_col='pred_agg_rf_ALL_fb',
               out_col='pred_agg_rf_ALL_fb2')
expiry_df_test_preds_g3 <- fallback_app_1(test_data_op=expiry_df_test_preds_g3,
               in_col='pred_agg_glm_ALL_fb',
               out_col='pred_agg_glm_ALL_fb2')

dim(expiry_df_test_preds_g2)
dim(expiry_df_test_preds_g3)
head(expiry_df_test_preds_g3)

[1] 932347     49

[1] 932347     59

,reseller,reseller_country,domain_id,renewal_type,renewed_count,expiry_date,creation_date,status,domain,registrar,registrant_country,reg_period,reg_revenue,reg_arpt,renew_mbg,renewal_item_id,renew_type,autorenew_type,renewal_status,renew_date,renew_arpt,gibb_score,pattern,pattern_domain_count,day_domains,⋯,pred_agg_glm,pred_agg_rf,pred_seg_glm_ALL,pred_seg_rf_ALL,pred_seg2_glm_ALL,pred_seg2_rf_ALL,reseller_geo,pred_agg_glm_ALL_fb,pred_agg_rf_ALL_fb,pred_agg_glm_fb,pred_agg_rf_fb,pred_seg_glm_ALL_fb,pred_seg_rf_ALL_fb,pred_seg2_glm_ALL_fb,pred_seg2_rf_ALL_fb,reg_arpt_org,reg_arpt_slab,pred_seg2_rf_ALL_fb2,pred_seg2_glm_ALL_fb2,pred_seg_rf_ALL_fb2,pred_seg_glm_ALL_fb2,pred_agg_rf_fb2,pred_agg_glm_fb2,pred_agg_rf_ALL_fb2,pred_agg_glm_ALL_fb2
,<fct>,<fct>,<int>,<fct>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<dbl>,<dbl>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<int>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,,,122049036,FirstTime,1,2020-08-30,2019-08-30,Active,qianxiyun.online,,NA,1,25,25.00,0,341703158,auto-renewal,unrealized,Not Renewd,2020-08-30,6,0.63,qianxiyun,1,1,⋯,NA,0.7713468,NA,NA,NA,NA,NA,NA,0.9550391,NA,0.7713468,NA,NA,NA,NA,25.00,"(15,25]",0.5510000,0.5510000,0.5510000,0.551,0.7713468,0.551,0.9550391,0.551
2,1&1 Internet,Germany,43450963,FirstTime,1,2020-01-27,2017-01-27,Active,ncyc.online,1&1 Internet,NA,3,55,18.33,0,239731665,transfer,NA,Renewed,2019-10-13,25,1.18,ncyc,1,23,⋯,0.817,0.6676114,0.543,0.7268116,0.5644593,0.7075837,EU,0.817,0.6088277,0.817,0.6676114,0.543,0.7268116,0.5644593,0.7075837,18.33,"(15,25]",0.7075837,0.5644593,0.7268116,0.543,0.6676114,0.817,0.6088277,0.817
3,1&1 Internet,Germany,43576888,FirstTime,1,2020-01-31,2017-01-31,Active,audev.online,1&1 Internet,NA,3,55,18.33,0,287997921,auto-renewal,realized,Renewed,2020-01-31,25,2.57,audev,1,43,⋯,0.814,0.6877335,0.536,0.7155560,0.5601541,0.7314602,EU,0.814,0.6477227,0.814,0.6877335,0.536,0.7155560,0.5601541,0.7314602,18.33,"(15,25]",0.7314602,0.5601541,0.7155560,0.536,0.6877335,0.814,0.6477227,0.814
4,1&1 Internet,Germany,43727452,FirstTime,1,2020-02-03,2017-02-03,Active,pinkpanther.store,1&1 Internet,NA,3,88,29.33,0,288786056,auto-renewal,unrealized,Not Renewd,2020-02-03,40,7.27,pinkpanther,1,3,⋯,0.838,0.2588170,0.513,0.2754756,0.1642908,0.1535971,EU,0.833,0.2384536,0.838,0.2588170,0.513,0.2754756,0.1642908,0.1535971,29.33,"(25,35]",0.1535971,0.1642908,0.2754756,0.513,0.2588170,0.838,0.2384536,0.833
5,1&1 Internet,Germany,46911148,FirstTime,1,2020-04-17,2017-04-17,Active,lmengineeringservices.online,1&1 Internet,NA,3,50,16.67,0,308749164,auto-renewal,realized,Renewed,2020-04-17,25,7.81,lmengineeringservices,1,69,⋯,0.775,0.3878439,0.468,0.3962124,0.4979086,0.3769304,EU,0.772,0.3972606,0.775,0.3878439,0.468,0.3962124,0.4979086,0.3769304,16.67,"(15,25]",0.3769304,0.4979086,0.3962124,0.468,0.3878439,0.775,0.3972606,0.772
6,1&1 Internet,Germany,47605334,FirstTime,1,2020-05-08,2017-05-08,Active,eagleeyeview.store,1&1 Internet,NA,3,90,30.00,0,313867134,auto-renewal,unrealized,Not Renewd,2020-05-08,40,3.41,eagleeyeview,1,4,⋯,0.829,0.1730035,0.516,0.1675698,0.1723292,0.1027711,EU,0.823,0.1925156,0.829,0.1730035,0.516,0.1675698,0.1723292,0.1027711,30.00,"(25,35]",0.1027711,0.1723292,0.1675698,0.516,0.1730035,0.829,0.1925156,0.823


In [63]:
# WRITE to CSV
write.csv(expiry_df_test_preds_g3, "../../data/output/datapull_20201116/expiry_df_test_preds_fb.csv",
         row.names=FALSE)


In [ ]:
system('gsutil cp -r /home/jupyter/Domains_202003/data/output/datapull_20201116 gs://output_shared/')